deeply refered on  
https://github.com/bentrevett/pytorch-sentiment-analysis/blob/master/1%20-%20Simple%20Sentiment%20Analysis.ipynb

In [1]:
import torch.nn as nn
import torch
from torch.autograd import Variable
import torch.nn.functional as F
import torch.optim as optim
from konlpy.tag import Okt
okt = Okt()

In [2]:
from torchtext.data import Field
from torchtext.data import TabularDataset
from torchtext.data import Iterator, BucketIterator

In [3]:
import pandas as pd

## Declaring the Fields
- Torchtext 는 데이터를 가져오는 과정에서 선언하는 방식을 사용합니다.
    - 데이터가 어떤 형식을 지닐 것인지에 대한 것에 대해 선언을 해주고 이에 따라 torchtext 는 데이터를 로딩합니다.

In [4]:
SEED = 1234

torch.manual_seed(SEED)
torch.cuda.manual_seed(SEED)
torch.backends.cudnn.deterministic = True

tokenize = lambda x: okt.nouns(x)

# TEXT = Field(tokenize=tokenize,use_vocab=True, include_lengths=False,batch_first=True,fix_length=20) 
# TEXT = Field(tokenize=tokenize) 
TEXT = Field(tokenize=tokenize,use_vocab=True) 
LABEL = Field(sequential=False,unk_token=None, use_vocab=True,dtype=torch.float)

## Constructing the Dataset
- fields 객체는 raw data 를 어떻게 가져올 지에 대한 선언이 담겨있습니다.
- TabularDataset 객체를 통해서, 어디서, 어떤 데이터를 가져올 지에 대해 선언을 해줍니다.
- 아래의 소스코드를 통해 형성된 객체는 generator 의 형태를 띕니다.

In [5]:
%%time 

datafields = [("X", TEXT), ("y", LABEL)]

train_data,test_data = TabularDataset.splits(
                            path=".", 
                            train='train_df.csv',
                            test='test_df.csv',
                            format='csv',
                            skip_header=True,
                            fields=datafields)

CPU times: user 9min 19s, sys: 5.02 s, total: 9min 24s
Wall time: 6min 50s


In [6]:
TEXT.build_vocab(train_data);LABEL.build_vocab(train_data)
TEXT.build_vocab(test_data);LABEL.build_vocab(test_data)

- TabularDatset 을 통해서, tokenizing 까지는 되었지만, word_to_integar process는 아직 이뤄지지 않았습니다. 
- 우리의 경우, train , text 데이터 셋에 대해서 TEXT 부분에 대해서, word_to_integar converting이 필요합니다.
- `TEXT.build_vocab(trn)` 이라는 코드를 통해, converting이 가능합니다.
- 위의 연산은 모든 training set에 있는 모든 엘리먼트들을 torchtext로 만들어줍니다. Torchtext는 vocabulary를 핸들링하는 Vocab이라는 클래스를 가지고 있습니다. Vocab클래스는 word와 id를 stoi attribute에서 mapping 시켜주고, itos attribute에서는 reverse mapping시켜줍니다.
- stoi : word_to_idx default dictionary 
- itos : word list

In [9]:
BATCH_SIZE = 64
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = 'cpu'
train_iter, test_iter = BucketIterator.splits(datasets=(train_data,test_data),
                                            batch_sizes=(BATCH_SIZE,BATCH_SIZE),  
                                            sort_key=lambda x: len(x.X), 
                                            device=device,
                                            sort_within_batch=True, 
                                            repeat=False)

In [73]:
class CNN(nn.Module) : 
    
    def __init__(self,VOCAB_SIZE , EMBED_SIZE , HID_SIZE , DROPOUT ,KERNEL_SIZE , NUM_FILTER , N_CLASS ) : 
        super(CNN, self).__init__()
        self.vocab_size = VOCAB_SIZE # 사용하는 전체 vocab의 갯수가 됩니다.
        self.embed_size = EMBED_SIZE # Embedding dimension 의 크기로 파라미터입니다.
        self.hid_size = HID_SIZE # Hidden layer의 dimension으로 이 또한 파라미터입니다.
        self.dropout = DROPOUT # dropout의 probability를 명시해줍니다.
        if type(KERNEL_SIZE) !=list :
            self.kernel_size = list(KERNEL_SIZE) # kernel의 사이즈로, 여러개의 kernel_size를 리스트 형태로 넣어줄 수 있습니다.
        else : self.kernel_size = KERNEL_SIZE # 많을 수록 complex해집니다.
        self.num_filter = NUM_FILTER # 각각의 kernel 이 몇 개씩 있는지에 대한 파라미터입니다. 많을 수록 complex해집니다.
        self.num_class = N_CLASS # output_dimension에 대한 argument로 sigmoid 값을 출력하기 때문에, 1로 합니다.
#         self.device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
        self.device = 'cpu'
        
        self.embedding = nn.Embedding(
            num_embeddings = self.vocab_size,
            embedding_dim = self.embed_size,
            padding_idx = 1) 

        # padding_idx 의 argument 에 1을 넣으면 pad 토큰은 training 에서 제외됩니다.
        # embedding vector's dimension : vocab_size * embed_size , LOOKUP TABLE 과 같습니다.
        
        self.convs = nn.ModuleList([(nn.Conv2d(in_channels = 1,out_channels = self.num_filter,\
        kernel_size = (kernel,self.embed_size))) for kernel in self.kernel_size])
        # in_channels : computer_vision에서는 rgb 이미지의 경우 in_channels 를 3으로 쓰곤 하지만, nlp에서는 1을 주로 사용합니다.
        # out_channels : input data에 대해서 더 많은 특징들을 잡아낼 수도 있는 잠재성을 가지고 있습니다. 즉, 파라미터 선택의 문제가 됩니다.
        # kernel_size : 합성곱 커널의 사이즈가 됩니다.
        
        self.fully_connect = nn.Sequential(
        nn.Linear(self.num_filter * len(self.kernel_size),self.hid_size),nn.ReLU(),
        nn.Dropout(self.dropout),nn.Linear(self.hid_size , self.num_class),
        )
        
        # 1. nn.Linear : concatenate 해준 num_filter * len(kernel_size) dimensional vector에 hidden layer를 곱해준다.
        # 2. nn.ReLU : non-linearity 를 보장해주기 위해, activation function 인 Relu를 집어넣어준다.
        # 3. nn.Dropout : self.dropout 확률로, dropout 을 적용해준다.
        # 4. nn.Linear : 마지막 출력값을 n-class classification 과 맞춰준 뒤, softmax or sigmoid etc 를 적용해준다.
    
    def forward(self,x) : 
        # x's dimension : [max_length, batch_size], 위에서 tokenizer func 를 통해, 30이 되어있을 것입니다.
        if len(x.shape) == 1 :
            x.unsqueeze_(0) 
        # [1, max_length, batch_size] 0차원에 1 dim 을 squeeze 하는 이유는, embedding 연산을 함께 하기 위함입니다.
        
        embed = self.embedding(x) #[max_length, batch_size, embedding_dim]
        embed = embed.unsqueeze(1) # [max_length, 1, batch_size, embedding_dim], convolution 을 위함입니다.
        embed = embed.permute(2,1,0,3)
        # [batch_size, 1 , max_length , embedding_dim]
        # 직육면체가 batch_size 만큼 있다고 생각하면 됩니다. kernel의 depth 는 존재하지 않기 때문에,
        # 또한 이전에 이 정보를, in_channels 에 매겨주었기 때문에, unsqueeze_로 넣어줍니다.
        
        
        convolution = [conv(embed).squeeze(3) for conv in self.convs]
        # [batch_size, num_filter, dimension after convolution(stride 값에 따라 변화할 수 있습니다!)]
        
        # nlp 에서 convolution 을 돌리면, feature_map 의 column 크기(길이)가 embedding dimension 과 같기 때문에, 1이 됩니다. 
        # embedding dimension 의 dimension index는 3이였기 때문에, 3번째 인덱스를 squeeze 해줍니다.
        # 또한 max_length 의 길이를 가졌던 2번째 인덱스가 kernel size 로 인해, 그 수가 변하고. 이 값은 stride 크기에 따라 변합니다.
        # 직사각행렬이 batch_size 만큼 있다고 하기 보단, 벡터가 num_filter 만큼 있고, 
        # 그러한 벡터'들'이 (벡터들인 이유는 convolution 이후에는 하나의 데이터 셋에 대해서 벡터 형태로 값이 나오게 되고,  
        # num_kernels 파라미터에 따라 그러한 벡터들이 여러 개가 나오게 되는데, 하나의 데이터 셋의 벡터는 차원은 같지만, 커널의 값은 
        # 달라서, 벡터의 엘리먼트 값은 다르게 됩니다. 말이 길어졌지만, 벡터들이 num_kernels만큼 나오게 됩니다.)
        #이러한 데이터 셋이 배치에 따라 한번에 뭉텅이씩 돌기 때문에, batch_size 만큼 있다고 생각하는 것이 보다 바람직한 것 같습니다.
        
        
        pooled = [F.max_pool1d(conv,(conv.size(2))).squeeze(2) for conv in convolution]
        # [batch_size, num_filter]
        # max_polling 은, 같은 kernel size 를 가지는, 동일한 filter에 대해 적용됩니다.
        # 따라서 pooling 을 해줄때, 두 번째 파라미터로 conv.size(2) 즉, 벡터의 길이에 해주는 것이죠.
        # 이에 따라, 벡터값이 max값만 추출, squeeze 를 통해서, 하나의 벡터는 하나의 스칼라가 된다. 즉, 1차원이 됩니다.
        
        dropout = [F.dropout(pool,self.dropout) for pool in pooled]
        # 두 번째 인자 dropout 은 확률값의 형태이며, epoch 가 바뀔 때마다, 은닉층의 뉴런을 dropout % 을 dropout 한다.
        # CNN 과 같이 parameter 의 갯수가 많은 complex 한 모델에는 꼭 들어가는 프로세스입니다.
        
        concatenate = torch.cat(dropout, dim = 1) 
        # [batch_size , num_filter * num_kernel]
        #concatenate 해서 NN을 돌려준다. kernel끼리 concat 시킵니다. 띠라서 하나의 데이터셋은 하나의 기다란 벡터를 가지게 됩니다.
        # 끝이 난 것 같지만, 비선형성을 추가해주어야 하고, output_dimension 또한 linear transform 으로 변환해주어야 합니다.
        
        logit = self.fully_connect(concatenate)
        # NN layer 에 넣어주는 과정입니다. concat 의 차원에 맞춰 linear layer 에 넣어주고, 
        #비선형성을 위해 relu activation function 에 넣어준 후, dropout 을 거쳐
        # 마지막 값은 num_of_class 같의 벡터가 나오게 됩니다.
        # binary classfication 이면 길이가 2인 벡터
        
        return logit

In [66]:
class fit() : 
    
    def __init__(self, model, train_iter, test_iter, epoch = 5) : 
        self.optimizer = optim.SGD(model.parameters(), lr=1e-3)
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model = model.to(device)
        self.criterion = nn.BCEWithLogitsLoss().to(device)
        self.train_iter = train_iter
        self.test_iter = test_iter
        self.epoch = epoch
        
    def binary_accuracy(self, preds, y):
        """
        Returns accuracy per batch, i.e. if you get 8/10 right, this returns 0.8, NOT 8
        """

        rounded_preds = torch.round(torch.sigmoid(preds))
        # model 에서는 fully connecting 만 시켰기 때문에, sigmoid 를 씌워서, rounding을 시켜줍니다.
        correct = (rounded_preds == y).float() # 같으면 1, 다르면 0이 된다.
        acc = correct.sum()/len(correct) # 한 번의 배치에서 맞은 갯수의 비율 즉, accuracy가 된다!
        return acc
    
    def train(self, model, iterator):
    
        epoch_loss = 0 # loss per epoch
        epoch_acc = 0 # accuracy per epoch

        model.train()
        for batch in iterator:
            if batch.X.size(0) == 0 : continue #데이터가 없으면, continue 합니다. 위에서 padding을 manually 적용해주어서 상관 x
            self.optimizer.zero_grad() # optimizer 의 gradient 를 manually 하게 zero 로 초기화시켜주어야 합니다.

            predictions = model(batch.X).squeeze(1) # model 에서 나오는 fc 의 size 는 [batch_size,num_layer] 이기 때문            
            loss = self.criterion(predictions, batch.y) # calculating the loss 

            acc = self.binary_accuracy(predictions, batch.y) # return the accracuy in form of ratio

            loss.backward() # back propagation

            self.optimizer.step() # update the SGD

            epoch_loss += loss.item()
            epoch_acc += acc.item()

        return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
    def evaluate(self, model, iterator):
    
        epoch_loss = 0
        epoch_acc = 0

        model.eval() # stop the every change in gradient of model

        with torch.no_grad():
            
            for batch in iterator:
                if batch.X.size(0) == 0 : continue
                predictions = model(batch.X).squeeze(1)
                loss = self.criterion(predictions, batch.y)

                acc = self.binary_accuracy(predictions, batch.y)

                epoch_loss += loss.item()
                epoch_acc += acc.item()

        return epoch_loss / len(iterator), epoch_acc / len(iterator)
    
    def fit_by_iterate(self) : 
        
        for epoch in range(self.epoch+1):
            print('epoch : ',epoch+1,end='\r')
            train_loss, train_acc = self.train(self.model, self.train_iter)

        valid_loss, valid_acc = self.evaluate(self.model, self.test_iter)
        print(f'| Epoch: {epoch+1:02} | Train Loss: {train_loss:.3f} | Train Acc: {train_acc*100:.2f}% | Val. Loss: {valid_loss:.3f} | Val. Acc: {valid_acc*100:.2f}% |')

In [70]:
VOCAB_SIZE = len(TEXT.vocab)
EMBED_SIZE = 256
HID_SIZE = 128
DROPOUT = 0.5
KERNEL_SIZE = [2,3,4,5]
NUM_FILTER = 4
N_CLASS = 1

model = CNN(VOCAB_SIZE, EMBED_SIZE, HID_SIZE, DROPOUT, KERNEL_SIZE, NUM_FILTER, N_CLASS)
model

CNN(
  (embedding): Embedding(23044, 256, padding_idx=1)
  (convs): ModuleList(
    (0): Conv2d(1, 4, kernel_size=(2, 256), stride=(1, 1))
    (1): Conv2d(1, 4, kernel_size=(3, 256), stride=(1, 1))
    (2): Conv2d(1, 4, kernel_size=(4, 256), stride=(1, 1))
    (3): Conv2d(1, 4, kernel_size=(5, 256), stride=(1, 1))
  )
  (fully_connect): Sequential(
    (0): Linear(in_features=16, out_features=128, bias=True)
    (1): ReLU()
    (2): Dropout(p=0.5)
    (3): Linear(in_features=128, out_features=1, bias=True)
  )
)

In [71]:
fitting_process = fit(model,train_iter,test_iter,epoch=5)

In [72]:
fitting_process.fit_by_iterate()

| Epoch: 05 | Train Loss: 0.639 | Train Acc: 62.87% | Val. Loss: 0.628 | Val. Acc: 64.05% |
